In [68]:
import pycatch22
import os
import scipy
import numpy as np
from sklearn.impute import SimpleImputer


In [3]:
ecg_data = os.path.join(os.getcwd(), 'train_data')


In [52]:
features = []

mat_files = []
for root, dirs, files in os.walk(ecg_data):
    for file in files:
        if file.endswith('.mat'):
            mat_files.append(os.path.join(root, file))

In [53]:

for file in mat_files:
    mat_data = scipy.io.loadmat(file)
    data_matrix = mat_data['my_matrix']

    results = pycatch22.catch22_all(data_matrix[0])

    features.append(results['values'])

In [9]:
for feature, output in zip(results['names'], results['values']):
    print(f"{feature}: {output}")

DN_HistogramMode_5: -1.0963774807251676
DN_HistogramMode_10: -0.6606201800043661
CO_f1ecac: 10.413233765269522
CO_FirstMin_ac: 42
CO_HistogramAMI_even_2_5: 0.5904441024313102
CO_trev_1_num: 0.003180337717153423
MD_hrv_classic_pnn40: 0.4346869373874775
SB_BinaryStats_mean_longstretch1: 168.0
SB_TransitionMatrix_3ac_sumdiagcov: 0.001953125
PD_PeriodicityWang_th0_01: 60
CO_Embed2_Dist_tau_d_expfit_meandiff: 0.9497900968869051
IN_AutoMutualInfoStats_40_gaussian_fmmi: 40.0
FC_LocalSimple_mean1_tauresrat: 0.046052631578947366
DN_OutlierInclude_p_001_mdrmd: 0.07279999999999998
DN_OutlierInclude_n_001_mdrmd: -0.1199
SP_Summaries_welch_rect_area_5_1: 0.99975914113523
SB_BinaryStats_diff_longstretch0: 35.0
SB_MotifThree_quantile_hh: 1.3311835606815494
SC_FluctAnal_2_rsrangefit_50_1_logi_prop_r1: 0.2653061224489796
SC_FluctAnal_2_dfa_50_1_2_logi_prop_r1: 0.2653061224489796
SP_Summaries_welch_rect_centroid: 0.09357282806103027
FC_LocalSimple_mean3_stderr: 0.27813252266939875


In [10]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

In [61]:
features = np.array(features)

labels = []

mapping_df = pd.read_csv('files_and_conditions1.csv')

mapping_dict = dict(zip(mapping_df['Name'], mapping_df['Condition']))


for file in mat_files:
    base_name = os.path.splitext(os.path.basename(file))[0]

    labels.append(mapping_dict.get(base_name))
labels = np.array(labels)
print(labels)


['Sinus Bradycardia' 'Sinus Irregularity' 'Normal' ... 'Normal' 'Normal'
 'Atrial Flutter']


In [70]:
print("Checking for NaNs in features:")
print(np.isnan(features).sum())

imputer = SimpleImputer(strategy='mean')
features = imputer.fit_transform(features)


Checking for NaNs in features:
0
0


In [73]:
print("Features dtype:", features.dtype)


Features dtype: float64


In [77]:
# Check for None values in labels
print("None in labels:", any(x is None for x in labels))
labels = [0 if x is None else x for x in labels]
print("None in labels:", any(x is None for x in labels))



None in labels: False
None in labels: False


In [78]:

scaler = StandardScaler()
features = scaler.fit_transform(features)

clf = SVC(kernel='linear') 
clf.fit(features, labels)

y_pred = clf.predict(features)
print(classification_report(labels, y_pred))

/Users/matildacotton/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


                                           precision    recall  f1-score   support

                                        0       0.00      0.00      0.00         1
          1 degree atrioventricular block       0.00      0.00      0.00       286
          2 degree atrioventricular block       0.00      0.00      0.00        43
2 degree atrioventricular block(Type one)       0.00      0.00      0.00        10
          3 degree atrioventricular block       0.00      0.00      0.00        12
                      Atrial Fibrillation       0.00      0.00      0.00       333
                           Atrial Flutter       0.00      0.00      0.00      2092
                       Atrial Tachycardia       0.00      0.00      0.00        45
   Atrioventricular Reentrant Tachycardia       0.00      0.00      0.00         4
                          Axis left shift       0.00      0.00      0.00       665
                         Axis right shift       0.00      0.00      0.00       156
   

/Users/matildacotton/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/matildacotton/anaconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
